In [1]:
from PIL import Image, ImageDraw
from io import BytesIO
import json
import joblib
import os
import random

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset

from sklearn.manifold import TSNE
import matplotlib.cm as cm
import matplotlib.pyplot as plt
plt_style = 'seaborn-talk'

random.seed(25)

In [2]:
lt_data = torch.load('/home/kaihua/projects/Generalized-Long-Tail.private/checkpoints_v/imagenet_lt_resnext_baseline/save_outputs_test_lt.pth')
bl_data = torch.load('/home/kaihua/projects/Generalized-Long-Tail.private/checkpoints_v/imagenet_lt_resnext_baseline/save_outputs_test_bl.pth')
gbl_data = torch.load('/home/kaihua/projects/Generalized-Long-Tail.private/checkpoints_v/imagenet_lt_resnext_baseline/save_outputs_test_bbl.pth')

In [67]:
def get_similarity(data):
    features = data['save_features']
    labels = data['save_labs']
    
    mean_features = []
    for i in range(1000):
        center = features[labels==i].mean(0).view(-1)
        center = center / (center * center).sum().sqrt()
        mean_features.append(center)
        
    similarity = []
    for i in range(features.shape[0]):
        feature = features[i] / (features[i] * features[i]).sum().sqrt()
        center = mean_features[labels[i].item()]
        sim = (feature * center).sum() 
        similarity.append(sim.item())
    
    return similarity

In [59]:
def get_distance(data):
    features = data['save_features']
    labels = data['save_labs']
    
    mean_features = []
    for i in range(1000):
        mean_features.append(features[labels==i].mean(0).view(-1))
        
    distances = []
    for i in range(features.shape[0]):
        dist = ((features[i] - mean_features[labels[i].item()]) ** 2).sum().item()
        distances.append(dist)
    
    distances = torch.FloatTensor(distances).sqrt()
    #distances = distances - distances.min()
    #distances = distances / distances.max()
    distances = distances.tolist()
    return distances

In [72]:
def get_prob(data):
    preds = data['save_preds']
    labels = data['save_labs']
    confidence = []
    for i in range(preds.shape[0]):
        confidence.append(preds[i, labels[i].item()].item())
    return confidence

In [47]:
bl_data['save_preds'].shape

torch.Size([60000, 1000])

In [51]:
distances = get_distance(lt_data)
for item in distances[1:20000]:
    print(item)

21.64569091796875
22.689542770385742
19.413476943969727
20.688657760620117
25.718427658081055
22.53723907470703
17.31264305114746
24.503286361694336
30.695209503173828
23.822534561157227
26.028364181518555
28.068517684936523
27.07587242126465
14.747514724731445
22.883047103881836
24.546133041381836
20.785961151123047
17.17488670349121
16.094432830810547
28.710342407226562
24.31684684753418
27.25577163696289
23.79482078552246
24.031749725341797
18.75312042236328
25.96165657043457
23.735551834106445
25.461109161376953
19.43268394470215
21.604663848876953
21.69505500793457
21.60619354248047
30.094249725341797
21.563194274902344
23.108016967773438
18.597631454467773
26.399900436401367
19.280296325683594
22.77137565612793
18.765151977539062
22.624547958374023
23.99913787841797
22.983076095581055
23.808290481567383
19.71208381652832
20.31631851196289
23.105073928833008
30.020998001098633
26.458881378173828
25.961395263671875
28.179183959960938
27.45343017578125
21.204072952270508
17.92513656

In [73]:
confidence = get_prob(lt_data)

In [74]:
for item in confidence[1:20000]:
    print(item)

5.333958625793457
13.690914154052734
14.61142635345459
12.495258331298828
6.963313102722168
14.801663398742676
10.44019889831543
10.69755744934082
3.236064910888672
16.822465896606445
8.546792984008789
1.2981462478637695
12.059307098388672
18.836688995361328
18.097469329833984
8.25714111328125
8.962047576904297
13.083017349243164
11.317138671875
9.245389938354492
4.177589416503906
7.354314804077148
5.046579360961914
16.712539672851562
12.76739501953125
9.282569885253906
15.222537994384766
8.629678726196289
18.054649353027344
17.108543395996094
14.562862396240234
8.480611801147461
12.787338256835938
13.07375431060791
7.9708709716796875
14.465567588806152
1.0006864070892334
12.74848461151123
7.816766738891602
14.603839874267578
14.112340927124023
8.66661262512207
14.046476364135742
19.21558952331543
14.374653816223145
16.221954345703125
7.462445259094238
0.5453565716743469
13.30124568939209
12.86134147644043
5.819326400756836
3.400099754333496
10.817120552062988
13.379125595092773
14.223

In [68]:
distances = get_similarity(lt_data)
for item in distances[1:20000]:
    print(item)

0.8529207706451416
0.7689545154571533
0.8484206199645996
0.8074488639831543
0.7794674634933472
0.8164239525794983
0.8281294107437134
0.8078876733779907
0.7106229066848755
0.7827877998352051
0.7519901990890503
0.7360295653343201
0.7354472875595093
0.8842880725860596
0.8602414727210999
0.6452043056488037
0.818099856376648
0.85528564453125
0.875069797039032
0.7650066018104553
0.757396399974823
0.7605969905853271
0.7606066465377808
0.7963001728057861
0.8651797771453857
0.7665569186210632
0.8327844142913818
0.7154766917228699
0.8692499995231628
0.8379287719726562
0.8130005598068237
0.7960137128829956
0.763118326663971
0.8220351934432983
0.7504895925521851
0.8584480881690979
0.7285346388816833
0.8212041854858398
0.7992793917655945
0.8570535182952881
0.7992917895317078
0.8082927465438843
0.7921110987663269
0.8387888669967651
0.8098245859146118
0.8444521427154541
0.7684648036956787
0.6680514216423035
0.7786584496498108
0.7798045873641968
0.707522988319397
0.7183361649513245
0.7936155796051025
